In [60]:
import pandas as pd
import os
import time
import shutil
from pathlib import Path
from typing import List, Tuple, Dict

class ChineseSyllableSynthesizerGTTS:
    """
    Chinese syllable synthesizer using Google Text-to-Speech (gTTS)
    """
    def __init__(self, output_dir: str):
        try:
            from gtts import gTTS
            self.gTTS = gTTS
            self.available = True
        except ImportError:
            print("gTTS not installed. Install with: pip install gtts")
            self.available = False

        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

    def synthesize_syllable(self, syllable: str, cache_dir: Path) -> str:
        """Synthesize a single syllable using gTTS and save in cache_dir."""
        if not self.available:
            return None

        import re
        safe_syllable = re.sub(r'[^\w\-_.]', '_', syllable)
        filename = f"{safe_syllable}.mp3"
        filepath = cache_dir / filename

        # Skip if already exists
        if filepath.exists():
            print(f"Using cached: {filepath}")
            return str(filepath)

        try:
            tts = self.gTTS(text=syllable, lang='zh', slow=False, tld='com')
            tts.save(str(filepath))
            print(f"Generated: {filepath}")
            return str(filepath)
        except Exception as e:
            print(f"Error generating {syllable}: {e}")
            return None

def process_trials_from_csv(
    csv_path: str,
    output_base: str = "/Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words",
    method: str = 'gtts'
):
    """
    Read a CSV where each row is a trial with columns:
    W1,W2,W3,...,Cue_Word,Cue_Pos,Cue,Block

    For each block, create a block folder. For each trial, create a trial folder under the block.
    In each trial folder, create 'words' and 'cue' subfolders.
    'words' stores every single character audio, 'cue' only stores cue audio.
    
    Strategy: First generate all unique syllables to a cache folder, then copy them to trial folders.
    """
    # Read CSV
    try:
        df = pd.read_csv(csv_path, encoding='utf-8')
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return None

    # Find all word columns (W1, W2, W3, ..., Wn)
    word_cols = [col for col in df.columns if col.startswith('W') and col[1:].isdigit()]
    # Required columns for cue and block
    required_cols = ['Cue_Word', 'Cue', 'Cue_Pos', 'block']
    for col in required_cols:
        if col not in df.columns:
            print(f"Column '{col}' not found. Available: {df.columns.tolist()}")
            return None

    # Initialize synthesizer
    if method == 'gtts':
        synthesizer = ChineseSyllableSynthesizerGTTS(output_base)
        if not synthesizer.available:
            return None
    else:
        print(f"Only 'gtts' method is implemented in this script.")
        return None

    # Step 1: Collect all unique syllables from the CSV
    print("\n=== Step 1: Collecting unique syllables ===")
    unique_syllables = set()
    for col in word_cols:
        for word in df[col]:
            word_str = str(word).strip()
            if word_str and word_str != 'nan':
                for char in word_str:
                    if char.strip():
                        unique_syllables.add(char)
    
    print(f"Found {len(unique_syllables)} unique syllables")
    
    # Step 2: Generate all syllables to a cache directory
    print("\n=== Step 2: Generating all syllable audio files ===")
    cache_dir = Path(output_base) / "_syllable_cache"
    cache_dir.mkdir(parents=True, exist_ok=True)
    
    syllable_cache: Dict[str, str] = {}
    for syllable in sorted(unique_syllables):
        audio_path = synthesizer.synthesize_syllable(syllable, cache_dir)
        if audio_path:
            syllable_cache[syllable] = audio_path
        time.sleep(0.5)  # Rate limiting
    
    print(f"Generated {len(syllable_cache)} syllable audio files")

    # Step 3: Process each trial and copy audio files from cache
    print("\n=== Step 3: Organizing audio files into trial folders ===")
    
    # Group by block
    for block, block_df in df.groupby('block'):
        block_dir = Path(output_base) / f"block_{block}"
        block_dir.mkdir(parents=True, exist_ok=True)
        print(f"\n=== Processing block {block} ===")

        # trial index should start from 1 for each block
        for trial_idx, (row_index, row) in enumerate(block_df.iterrows(), start=1):
            trial_dir = block_dir / f"trial_{trial_idx}"
            words_dir = trial_dir / "words"
            cue_dir = trial_dir / "cue"
            words_dir.mkdir(parents=True, exist_ok=True)
            cue_dir.mkdir(parents=True, exist_ok=True)
            print(f"\n--- Processing trial {trial_idx} in block {block} ---")

            # Process all words (W1, W2, ..., Wn)
            for word_idx, col in enumerate(word_cols):
                word = str(row[col]).strip()
                if not word or word == 'nan':
                    print(f"Skipping empty word in {col}")
                    continue
                
                # Copy each syllable from cache
                syllables = list(word)
                for syllable_idx, syllable in enumerate(syllables):
                    if syllable.strip() and syllable in syllable_cache:
                        import re
                        safe_word = re.sub(r'[^\w\-_.]', '_', word)
                        filename = f"word{word_idx+1}_syllable_{syllable_idx+1}_{safe_word}_{syllable}.mp3"
                        src_path = Path(syllable_cache[syllable])
                        dst_path = words_dir / filename
                        shutil.copy2(src_path, dst_path)
                        print(f"Copied: {syllable} -> {dst_path}")

            # Process cue: use Cue_Word and Cue_Pos as integer indices
            cue_word_idx = None
            cue_char = None

            # Get cue_word_idx and cue_pos as integers
            try:
                cue_word_idx = int(row['Cue_Word']) - 1 if not pd.isnull(row['Cue_Word']) else None
                cue_pos = int(row['Cue_Pos']) if not pd.isnull(row['Cue_Pos']) else None
            except Exception as e:
                print(f"Error parsing Cue_Word or Cue_Pos for trial {trial_idx}: {e}")
                cue_word_idx = None
                cue_pos = None

            # Get the cue word and cue character
            cue_word = None
            if cue_word_idx is not None and 0 <= cue_word_idx < len(word_cols):
                cue_word_col = word_cols[cue_word_idx]
                cue_word = str(row[cue_word_col]).strip()
                if cue_word and cue_pos is not None and 0 <= cue_pos < len(cue_word):
                    cue_char = cue_word[cue_pos]
                else:
                    cue_char = None

            if cue_word and cue_char and cue_word_idx is not None and cue_pos is not None:
                # The audio file in words_dir is named as:
                # word{word_idx+1}_syllable_{syllable_index+1}_{safe_word}_{syllable}.mp3
                import re
                safe_word = re.sub(r'[^\w\-_.]', '_', cue_word)
                filename = f"word{cue_word_idx+1}_syllable_{cue_pos+1}_{safe_word}_{cue_char}.mp3"
                src_path = words_dir / filename
                dst_path = cue_dir / filename
                if src_path.exists():
                    shutil.copy2(src_path, dst_path)
                    print(f"Copied cue audio from {src_path} to {dst_path}")
                else:
                    print(f"Warning: Cue audio file {src_path} does not exist for trial {trial_idx}")
            else:
                print(f"Warning: Cue word or cue character missing or not found for trial {trial_idx}")

    print("\nAll trials processed. Audio files are in:", output_base)
    print(f"Syllable cache is in: {cache_dir}")

if __name__ == "__main__":
    # Example usage - choose one of the three CSV files:
    
    # Option 1: three_4_syllable_exp_with_blocks.csv
    # csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/three_4_syllable_exp_with_blocks.csv"
    # output_base = "/Users/yufang/WM_load/chinese_audio_output/three_4_syllable_words"
    
    # Option 2: three_3_syllable_exp_with_blocks.csv
    # csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/three_3_syllable_exp_with_blocks.csv"
    # output_base = "/Users/yufang/WM_load/chinese_audio_output/three_3_syllable_words"
    
    # Option 3: four_3_syllable_exp_with_blocks.csv
    csv_path = "Exp1_Syllable_Sequence_and_Cueing_Syllable_List/four_3_syllable_exp_with_blocks.csv"
    output_base = "/Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words"
    
    process_trials_from_csv(csv_path, output_base)


=== Step 1: Collecting unique syllables ===
Found 358 unique syllables

=== Step 2: Generating all syllable audio files ===
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/业.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/中.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/主.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/乐.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/乒.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/乓.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/乘.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache/习.mp3
Generated: /Users/yufang/WM_load/chinese_audio_output/four_3_syllable_words/_syllable_cache

In [ ]:
# import pandas as pd
# import os
# import time
# import shutil
# from pathlib import Path
# from typing import List, Tuple

# class ChineseSyllableSynthesizerGTTS:
#     """
#     Chinese syllable synthesizer using Google Text-to-Speech (gTTS)
#     """
#     def __init__(self, output_dir: str):
#         try:
#             from gtts import gTTS
#             self.gTTS = gTTS
#             self.available = True
#         except ImportError:
#             print("gTTS not installed. Install with: pip install gtts")
#             self.available = False

#         try:
#             from pydub import AudioSegment
#             # Set ffmpeg path for pydub if needed
#             AudioSegment.converter = "/opt/homebrew/bin/ffmpeg"  # Common macOS path
#             AudioSegment.ffmpeg = "/opt/homebrew/bin/ffmpeg"
#             AudioSegment.ffprobe = "/opt/homebrew/bin/ffprobe"
#             self.AudioSegment = AudioSegment
#             self.pydub_available = True
#         except ImportError:
#             print("pydub not installed. Install with: pip install pydub")
#             self.pydub_available = False
#         except Exception as e:
#             print(f"pydub available but ffmpeg/ffprobe not found: {e}")
#             print("You may need to install ffmpeg: brew install ffmpeg")
#             self.pydub_available = False

#         self.output_dir = Path(output_dir)
#         self.output_dir.mkdir(parents=True, exist_ok=True)

#     def synthesize_syllable(self, syllable: str, word: str, syllable_index: int, temp_dir: Path) -> str:
#         """Synthesize a single syllable using gTTS and save in temp_dir."""
#         if not self.available:
#             return None

#         import re
#         safe_word = re.sub(r'[^\w\-_.]', '_', word)
#         filename = f"temp_syllable_{syllable_index + 1}_{safe_word}_{syllable}.mp3"
#         filepath = temp_dir / filename

#         try:
#             tts = self.gTTS(text=syllable, lang='zh', slow=False, tld='com')
#             tts.save(str(filepath))
#             print(f"Generated: {filepath}")
#             return str(filepath)
#         except Exception as e:
#             print(f"Error generating {syllable}: {e}")
#             return None

#     def process_word(self, word: str, word_dir: Path) -> Tuple[List[str], str]:
#         """Process a single word into syllables, combine them into one audio file."""
#         syllables = list(word.strip())
        
#         # Create temporary directory for individual syllables
#         temp_dir = word_dir / "temp_syllables"
#         temp_dir.mkdir(parents=True, exist_ok=True)

#         print(f"Processing word: {word}")
#         print(f"Syllables: {syllables}")

#         # Generate individual syllable audio files
#         temp_audio_paths = []
#         for i, syllable in enumerate(syllables):
#             if syllable.strip():
#                 audio_path = self.synthesize_syllable(syllable, word, i, temp_dir)
#                 if audio_path:
#                     temp_audio_paths.append(audio_path)
#                 time.sleep(0.5)  # Rate limiting

#         # Try to combine all syllables into one audio file if pydub is available
#         if temp_audio_paths and self.pydub_available:
#             try:
#                 combined_audio = None
#                 for audio_path in temp_audio_paths:
#                     audio_segment = self.AudioSegment.from_mp3(audio_path)
#                     if combined_audio is None:
#                         combined_audio = audio_segment
#                     else:
#                         combined_audio += audio_segment
                
#                 # Save combined audio
#                 import re
#                 safe_word = re.sub(r'[^\w\-_.]', '_', word)
#                 combined_filename = f"word_{safe_word}.mp3"
#                 combined_filepath = word_dir / combined_filename
#                 combined_audio.export(str(combined_filepath), format="mp3")
#                 print(f"Combined audio saved: {combined_filepath}")
                
#                 # Keep temporary files in temp_syllables folder instead of deleting
#                 print(f"Individual syllable files kept in: {temp_dir}")
                
#                 return syllables, str(combined_filepath)
#             except Exception as e:
#                 print(f"Error combining audio files: {e}")
#                 print("Individual syllable files are still available in the temp_syllables directory.")
#                 print("To fix this, install ffmpeg: brew install ffmpeg (macOS) or apt-get install ffmpeg (Linux)")
#                 return syllables, None
#         elif temp_audio_paths:
#             print("pydub not available or ffmpeg not found. Individual syllable files generated in temp_syllables directory.")
#             return syllables, None
        
#         return syllables, None

# class AudioPlayer:
#     """
#     Audio player for testing existing audio files
#     """
#     def __init__(self):
#         try:
#             import pygame
#             pygame.mixer.init()
#             self.pygame = pygame
#             self.available = True
#         except ImportError:
#             print("pygame not installed. Install with: pip install pygame")
#             self.available = False

#     def play_audio(self, audio_path: str, wait_for_finish: bool = True):
#         """Play an audio file"""
#         if not self.available:
#             print(f"Cannot play {audio_path} - pygame not available")
#             return False
        
#         try:
#             self.pygame.mixer.music.load(audio_path)
#             self.pygame.mixer.music.play()
#             print(f"Playing: {audio_path}")
            
#             if wait_for_finish:
#                 while self.pygame.mixer.music.get_busy():
#                     self.pygame.time.wait(100)
            
#             return True
#         except Exception as e:
#             print(f"Error playing {audio_path}: {e}")
#             return False

# def process_words_from_txt(
#     txt_path: str = "/Users/yufang/WM_load/word_database_txt/three_4_syllable_words.txt",
#     output_base: str = "/Users/yufang/WM_load/test_audio/three_4_syllable",
#     method: str = 'gtts'
# ):
#     """
#     Read a text file where each line is a 4-syllable Chinese word.
#     For each word, create a folder and synthesize each syllable independently,
#     then combine them into one audio file per word.
#     """
#     # Read text file
#     try:
#         with open(txt_path, 'r', encoding='utf-8') as f:
#             words = [line.strip() for line in f if line.strip()]
#     except Exception as e:
#         print(f"Error reading text file: {e}")
#         return None

#     print(f"Found {len(words)} words to process")

#     # Initialize synthesizer
#     if method == 'gtts':
#         synthesizer = ChineseSyllableSynthesizerGTTS(output_base)
#         if not synthesizer.available:
#             print("gTTS is not available. Cannot proceed.")
#             return None
#         if not synthesizer.pydub_available:
#             print("Warning: pydub not available or ffmpeg not found. Individual syllables will be generated in temp_syllables folders.")
#     else:
#         print(f"Only 'gtts' method is implemented in this script.")
#         return None

#     # Process each word from word 1 to last
#     for word_idx, word in enumerate(words, start=1):
#         if not word:
#             continue
        
#         # Create a folder for this word
#         import re
#         safe_word = re.sub(r'[^\w\-_.]', '_', word)
#         word_dir = Path(output_base) / f"word_{word_idx}_{safe_word}"
#         word_dir.mkdir(parents=True, exist_ok=True)
        
#         print(f"\n=== Processing word {word_idx}/{len(words)}: {word} ===")
        
#         # Process the word (synthesize each syllable and combine)
#         syllables, combined_audio_path = synthesizer.process_word(word, word_dir)
        
#         if combined_audio_path:
#             print(f"Completed word {word_idx}: {word} with {len(syllables)} syllables combined into one file")
#             print(f"Individual syllables available in: {word_dir / 'temp_syllables'}")
#         else:
#             print(f"Completed word {word_idx}: {word} - individual syllables generated in temp_syllables folder")

#     print(f"\nAll words processed. Audio files are in: {output_base}")
#     print("Individual syllable files are in temp_syllables subfolders within each word folder.")

# def play_audio_from_folders(
#     audio_base_dir: str = "/Users/yufang/WM_load/test_audio/three_4_syllable",
#     play_delay: float = 1.0,
#     start_word_index: int = 1
# ):
#     """
#     Iterate through folders and play audio files one by one from a specified word index to last.
#     This option doesn't synthesize voice, just plays existing audio files.
#     Looks for audio files in both the main word folders and temp_syllables subfolders.
    
#     Args:
#         audio_base_dir: Base directory containing word folders with audio files
#         play_delay: Delay between playing different audio files (seconds)
#         start_word_index: Word index to start playing from (1-based, default is 1)
#     """
#     audio_base_path = Path(audio_base_dir)
    
#     if not audio_base_path.exists():
#         print(f"Audio directory does not exist: {audio_base_dir}")
#         return
    
#     # Initialize audio player
#     player = AudioPlayer()
#     if not player.available:
#         print("Audio player not available. Install pygame: pip install pygame")
#         return
    
#     # Find all word folders and sort them by word number (word_1, word_2, etc.)
#     word_folders = [d for d in audio_base_path.iterdir() if d.is_dir() and d.name.startswith('word_')]
    
#     # Sort by word number to ensure processing from word 1 to last
#     def extract_word_number(folder_name):
#         try:
#             # Extract number from folder name like "word_1_safe_word" or "word_123_safe_word"
#             parts = folder_name.split('_')
#             if len(parts) >= 2:
#                 return int(parts[1])
#             return 0
#         except:
#             return 0
    
#     word_folders.sort(key=lambda x: extract_word_number(x.name))
    
#     if not word_folders:
#         print(f"No word folders found in {audio_base_dir}")
#         return
    
#     # Filter folders to start from the specified word index
#     filtered_folders = [f for f in word_folders if extract_word_number(f.name) >= start_word_index]
    
#     if not filtered_folders:
#         print(f"No word folders found with index >= {start_word_index}")
#         return
    
#     print(f"Found {len(word_folders)} total word folders")
#     print(f"Starting from word index {start_word_index}, processing {len(filtered_folders)} folders")
    
#     for folder_idx, word_folder in enumerate(filtered_folders, start=1):
#         word_number = extract_word_number(word_folder.name)
#         print(f"\n=== Folder {folder_idx}/{len(filtered_folders)}: {word_folder.name} (Word {word_number}) ===")
        
#         # Find audio files in this folder and its temp_syllables subfolder
#         audio_files = []
        
#         # Check main folder
#         for ext in ['*.mp3', '*.wav', '*.m4a']:
#             audio_files.extend(word_folder.glob(ext))
        
#         # Check temp_syllables subfolder
#         temp_syllables_dir = word_folder / "temp_syllables"
#         if temp_syllables_dir.exists():
#             for ext in ['*.mp3', '*.wav', '*.m4a']:
#                 audio_files.extend(temp_syllables_dir.glob(ext))
        
#         if not audio_files:
#             print(f"No audio files found in {word_folder} or its temp_syllables subfolder")
#             continue
        
#         # Sort audio files by name
#         audio_files.sort()
        
#         print(f"Found {len(audio_files)} audio files:")
#         for audio_file in audio_files:
#             # Show relative path to indicate if it's in temp_syllables
#             relative_path = audio_file.relative_to(word_folder)
#             print(f"  - {relative_path}")
        
#         # Play each audio file
#         for audio_idx, audio_file in enumerate(audio_files, start=1):
#             relative_path = audio_file.relative_to(word_folder)
#             print(f"\nPlaying audio {audio_idx}/{len(audio_files)}: {relative_path}")
            
#             success = player.play_audio(str(audio_file), wait_for_finish=True)
            
#             if success:
#                 print(f"Finished playing: {relative_path}")
#             else:
#                 print(f"Failed to play: {relative_path}")
            
#             # Wait before playing next audio (except for the last one)
#             if audio_idx < len(audio_files):
#                 print(f"Waiting {play_delay} seconds before next audio...")
#                 time.sleep(play_delay)
        
#         # Wait before moving to next folder
#         if folder_idx < len(filtered_folders):
#             print(f"\nWaiting {play_delay * 2} seconds before next folder...")
#             time.sleep(play_delay * 2)
    
#     print("\nFinished playing all audio files!")

# if __name__ == "__main__":
#     import sys
#     sys.argv = ["", "play", "90"]
    
#     if len(sys.argv) > 1 and sys.argv[1] == "play":
#         # Play existing audio files
#         # Check if a start word index is provided
#         start_index = 1
#         if len(sys.argv) > 2:
#             try:
#                 start_index = int(sys.argv[2])
#                 print(f"Starting from word index: {start_index}")
#             except ValueError:
#                 print(f"Invalid start index '{sys.argv[2]}', using default (1)")
        
#         print("Playing existing audio files...")
#         play_audio_from_folders(start_word_index=start_index)
#     else:
#         # Process words from text file (synthesize new audio)
#         print("Synthesizing new audio files...")
#         process_words_from_txt()

Starting from word index: 90
Playing existing audio files...
Found 146 total word folders
Starting from word index 90, processing 57 folders

=== Folder 1/57: word_90_浩浩荡荡 (Word 90) ===
Found 4 audio files:
  - temp_syllables/temp_syllable_1_浩浩荡荡_浩.mp3
  - temp_syllables/temp_syllable_2_浩浩荡荡_浩.mp3
  - temp_syllables/temp_syllable_3_浩浩荡荡_荡.mp3
  - temp_syllables/temp_syllable_4_浩浩荡荡_荡.mp3

Playing audio 1/4: temp_syllables/temp_syllable_1_浩浩荡荡_浩.mp3
Playing: /Users/yufang/WM_load/test_audio/three_4_syllable/word_90_浩浩荡荡/temp_syllables/temp_syllable_1_浩浩荡荡_浩.mp3
Finished playing: temp_syllables/temp_syllable_1_浩浩荡荡_浩.mp3
Waiting 1.0 seconds before next audio...

Playing audio 2/4: temp_syllables/temp_syllable_2_浩浩荡荡_浩.mp3
Playing: /Users/yufang/WM_load/test_audio/three_4_syllable/word_90_浩浩荡荡/temp_syllables/temp_syllable_2_浩浩荡荡_浩.mp3
Finished playing: temp_syllables/temp_syllable_2_浩浩荡荡_浩.mp3
Waiting 1.0 seconds before next audio...

Playing audio 3/4: temp_syllables/temp_syllable_3_浩浩荡荡